In [ ]:
!pip install python-arango sentence-transformers datasets tqdm

In [2]:
import time
from arango import ArangoClient
from sentence_transformers import SentenceTransformer
from datasets import load_dataset
from tqdm import tqdm

In [3]:
ARANGO_CONFIG = {
    "hosts": "https://bfc25a0e3c74.arangodb.cloud:8529",
    "username": "root",
    "password": "VnicTWKeXaDasFNfmCfU",
    "db_name": "pubmed_graph",
    "chunk_col": "Chunks",
    "context_edge": "HAS_CONTEXT",
    "mention_edge": "MENTIONS"
}

In [ ]:
# @title 🚀 GraphRAG Builder (Fixed & Complete)
# This script installs dependencies, connects to ArangoDB, sets up the schema,
# and ingests the PubMedQA dataset into the graph.

# --- MANUAL CONFIGURATION ---
# Paste your details directly here to avoid input errors:

# 1. The URL must start with https:// and usually ends with :8529
ARANGO_URL = "https://bfc25a0e3c74.arangodb.cloud:8529"

# 2. The Username is almost always 'root'
ARANGO_USER = "root"

# 3. Paste the password you copied from the 'Users' tab
ARANGO_PASS = "VnicTWKeXaDasFNfmCfU"

# Database Name
DB_NAME = "pubmed_graph"

# --- CONNECT ---
print(f"Connecting to {ARANGO_URL}...")
client = ArangoClient(hosts=ARANGO_URL)
sys_db = client.db("_system", username=ARANGO_USER, password=ARANGO_PASS)

# Create/Connect to specific database
if not sys_db.has_database(DB_NAME):
    sys_db.create_database(DB_NAME)
    print(f"Created database: {DB_NAME}")
else:
    print(f"Using existing database: {DB_NAME}")

db = client.db(DB_NAME, username=ARANGO_USER, password=ARANGO_PASS)
print("✅ Connected Successfully!")

# --- SCHEMA SETUP ---
print("\nCreating Graph Schema...")

# 1. Define Node Collections
node_collections = ["Papers", "Chunks", "Concepts"]
for col in node_collections:
    if not db.has_collection(col):
        db.create_collection(col)
        print(f" - Created Node Collection: {col}")

# 2. Define Edge Collections
edge_collections = ["HAS_CONTEXT", "MENTIONS"]
for col in edge_collections:
    if not db.has_collection(col):
        db.create_collection(col, edge=True)
        print(f" - Created Edge Collection: {col}")

# 3. Create ArangoSearch View (Fallback for Vector Search)
# FIXED: The 'vector' index type is experimental in your version.
# We use an ArangoSearch View instead, which is robust and works on all versions.
view_name = "pubmed_view"

# FIXED: Use db.views() list comprehension to check existence instead of .has_view()
existing_views = [v["name"] for v in db.views()]

if view_name not in existing_views:
    # FIXED: Use dedicated method 'create_arangosearch_view' to avoid TypeError on 'type' arg
    db.create_arangosearch_view(
        name=view_name,
        properties={
            "links": {
                "Chunks": {
                    "fields": {
                        "embedding": {
                            "analyzers": ["identity"]  # Needed for vector operations
                        },
                        "text": {
                            "analyzers": ["text_en"]   # Useful for keyword search
                        }
                    }
                }
            }
        }
    )
    print(f" - Created ArangoSearch View: {view_name}")
else:
    print(f" - ArangoSearch View '{view_name}' already exists.")

print("\n✅ Database Configured Successfully!")

# --- LOAD DATA & MODEL ---
print("\nLoading Embedding Model & Dataset...")

# Load Model (Runs on GPU if available in Colab)
# We use all-MiniLM-L6-v2 for speed and good performance
model = SentenceTransformer('all-MiniLM-L6-v2')

# Load Dataset (Standard download to avoid 429 Rate Limits)
# REMOVED: streaming=True to prevent "Too Many Requests" error
ds = load_dataset("qiaojin/PubMedQA", "pqa_unlabeled", split="train")

print("✅ Model and Data ready.")

# --- PROCESSING LOOP ---
# This loop processes papers, chunks them, embeds them, and inserts into ArangoDB.

BATCH_SIZE = 50  # Number of papers to process before sending to DB (smaller batch for safety)
LIMIT_PAPERS = None # Limit for this run to ensure it finishes quickly (Set to None for full dataset)

papers_batch = []
chunks_batch = []
concepts_batch = []
edges_batch = []

print(f"\n🚀 Starting Ingestion (Limit: {LIMIT_PAPERS} papers)...")
start_time = time.time()

count = 0

for row in tqdm(ds, total=LIMIT_PAPERS):
    if LIMIT_PAPERS and count >= LIMIT_PAPERS:
        break

    pubid = row['pubid']
    question = row['question']
    long_answer = row['long_answer']

    # 1. Prepare Paper Node
    paper_key = str(pubid)
    papers_batch.append({
        "_key": paper_key,
        "title": question,
        "answer": long_answer
    })

    # 2. Process Concepts (MeSH Terms)
    mesh_terms = row.get('context', {}).get('meshes', [])
    for mesh in mesh_terms:
        # Sanitize key (Arango keys cannot contain spaces/special chars easily, so we hash or simplify)
        # Here we just remove non-alphanumeric for simplicity
        mesh_key = "".join(x for x in mesh if x.isalnum())
        if not mesh_key: continue

        # Add Concept Node
        concepts_batch.append({
            "_key": mesh_key,
            "name": mesh
        })

        # Link Paper -> Concept
        edges_batch.append({
            "_collection": "MENTIONS",
            "_from": f"Papers/{paper_key}",
            "_to": f"Concepts/{mesh_key}"
        })

    # 3. Process Contexts (Chunks)
    contexts = row.get('context', {}).get('contexts', [])
    labels = row.get('context', {}).get('labels', [])

    if contexts:
        # Embed all chunks for this paper at once
        embeddings = model.encode(contexts)

        for idx, (text, emb) in enumerate(zip(contexts, embeddings)):
            chunk_key = f"{paper_key}_{idx}"

            # Add Chunk Node
            chunks_batch.append({
                "_key": chunk_key,
                "text": text,
                "label": labels[idx] if idx < len(labels) else "context",
                "embedding": emb.tolist() # Convert numpy array to list for JSON
            })

            # Link Paper -> Chunk
            edges_batch.append({
                "_collection": "HAS_CONTEXT",
                "_from": f"Papers/{paper_key}",
                "_to": f"Chunks/{chunk_key}"
            })

    count += 1

    # --- BATCH INSERTION ---
    if count % BATCH_SIZE == 0:
        # Insert Papers
        if papers_batch:
            db.collection("Papers").import_bulk(papers_batch, on_duplicate="ignore")
        # Insert Concepts
        if concepts_batch:
            db.collection("Concepts").import_bulk(concepts_batch, on_duplicate="ignore")
        # Insert Chunks
        if chunks_batch:
            db.collection("Chunks").import_bulk(chunks_batch, on_duplicate="ignore")

        # Insert Edges (Must split by collection type for import_bulk)
        mentions = [e for e in edges_batch if e["_collection"] == "MENTIONS"]
        contexts = [e for e in edges_batch if e["_collection"] == "HAS_CONTEXT"]

        if mentions:
            db.collection("MENTIONS").import_bulk(mentions, on_duplicate="ignore")
        if contexts:
            db.collection("HAS_CONTEXT").import_bulk(contexts, on_duplicate="ignore")

        # Reset batches
        papers_batch = []
        chunks_batch = []
        concepts_batch = []
        edges_batch = []

# Final flush for remaining data
if papers_batch: db.collection("Papers").import_bulk(papers_batch, on_duplicate="ignore")
if concepts_batch: db.collection("Concepts").import_bulk(concepts_batch, on_duplicate="ignore")
if chunks_batch: db.collection("Chunks").import_bulk(chunks_batch, on_duplicate="ignore")

mentions = [e for e in edges_batch if e["_collection"] == "MENTIONS"]
contexts = [e for e in edges_batch if e["_collection"] == "HAS_CONTEXT"]
if mentions: db.collection("MENTIONS").import_bulk(mentions, on_duplicate="ignore")
if contexts: db.collection("HAS_CONTEXT").import_bulk(contexts, on_duplicate="ignore")

end_time = time.time()
print(f"\n🎉 Finished! Processed {count} papers in {end_time - start_time:.2f} seconds.")
print(f"Go to your ArangoDB Dashboard to see the 'pubmed_graph' database.")
print(f"IMPORTANT: Use 'FOR doc IN pubmed_view' in your AQL queries!")

In [ ]:
# @title ➕ Add PubMedQA "Labeled" Subset
# This script adds the 1,000 labeled papers to your existing graph.


# --- MANUAL CONFIGURATION ---
# 1. The URL must start with https:// and usually ends with :8529
ARANGO_URL = "https://bfc25a0e3c74.arangodb.cloud:8529"
# 2. The Username
ARANGO_USER = "root"
# 3. Paste the password you copied from the 'Users' tab
ARANGO_PASS = "VnicTWKeXaDasFNfmCfU"
# Database Name
DB_NAME = "pubmed_graph"

# --- CONNECT ---
print(f"Connecting to {ARANGO_URL}...")
client = ArangoClient(hosts=ARANGO_URL)
db = client.db(DB_NAME, username=ARANGO_USER, password=ARANGO_PASS)
print("✅ Connected to 'pubmed_graph'!")

# --- LOAD DATA & MODEL ---
print("\nLoading 'pqa_labeled' dataset...")

# Load the LABELED subset this time
ds_labeled = load_dataset("qiaojin/PubMedQA", "pqa_labeled", split="train")
model = SentenceTransformer('all-MiniLM-L6-v2')

print(f"✅ Loaded {len(ds_labeled)} labeled papers.")

# --- PROCESSING LOOP ---
BATCH_SIZE = 50
papers_batch = []
chunks_batch = []
concepts_batch = []
edges_batch = []

print("\n🚀 Starting Ingestion of Labeled Data...")
start_time = time.time()
count = 0

for row in tqdm(ds_labeled):
    pubid = row['pubid']
    question = row['question']
    long_answer = row['long_answer']
    final_decision = row.get('final_decision', None) # Unique to labeled set

    # 1. Prepare Paper Node (With extra 'final_decision' field)
    paper_key = str(pubid)
    papers_batch.append({
        "_key": paper_key,
        "title": question,
        "answer": long_answer,
        "decision": final_decision,  # Store 'yes', 'no', or 'maybe'
        "dataset": "labeled"         # Tag it so we know source
    })

    # 2. Process Concepts (MeSH Terms)
    mesh_terms = row.get('context', {}).get('meshes', [])
    for mesh in mesh_terms:
        mesh_key = "".join(x for x in mesh if x.isalnum())
        if not mesh_key: continue

        concepts_batch.append({
            "_key": mesh_key,
            "name": mesh
        })
        edges_batch.append({
            "_collection": "MENTIONS",
            "_from": f"Papers/{paper_key}",
            "_to": f"Concepts/{mesh_key}"
        })

    # 3. Process Contexts (Chunks)
    contexts = row.get('context', {}).get('contexts', [])
    labels = row.get('context', {}).get('labels', [])

    if contexts:
        embeddings = model.encode(contexts)
        for idx, (text, emb) in enumerate(zip(contexts, embeddings)):
            chunk_key = f"{paper_key}_{idx}"
            chunks_batch.append({
                "_key": chunk_key,
                "text": text,
                "label": labels[idx] if idx < len(labels) else "context",
                "embedding": emb.tolist()
            })
            edges_batch.append({
                "_collection": "HAS_CONTEXT",
                "_from": f"Papers/{paper_key}",
                "_to": f"Chunks/{chunk_key}"
            })

    count += 1

    # --- BATCH INSERTION ---
    if count % BATCH_SIZE == 0:
        if papers_batch: db.collection("Papers").import_bulk(papers_batch, on_duplicate="update") # Update if exists
        if concepts_batch: db.collection("Concepts").import_bulk(concepts_batch, on_duplicate="ignore")
        if chunks_batch: db.collection("Chunks").import_bulk(chunks_batch, on_duplicate="ignore")

        mentions = [e for e in edges_batch if e["_collection"] == "MENTIONS"]
        contexts = [e for e in edges_batch if e["_collection"] == "HAS_CONTEXT"]
        if mentions: db.collection("MENTIONS").import_bulk(mentions, on_duplicate="ignore")
        if contexts: db.collection("HAS_CONTEXT").import_bulk(contexts, on_duplicate="ignore")

        papers_batch = []
        chunks_batch = []
        concepts_batch = []
        edges_batch = []

# Final Flush
if papers_batch: db.collection("Papers").import_bulk(papers_batch, on_duplicate="update")
if concepts_batch: db.collection("Concepts").import_bulk(concepts_batch, on_duplicate="ignore")
if chunks_batch: db.collection("Chunks").import_bulk(chunks_batch, on_duplicate="ignore")
mentions = [e for e in edges_batch if e["_collection"] == "MENTIONS"]
contexts = [e for e in edges_batch if e["_collection"] == "HAS_CONTEXT"]
if mentions: db.collection("MENTIONS").import_bulk(mentions, on_duplicate="ignore")
if contexts: db.collection("HAS_CONTEXT").import_bulk(contexts, on_duplicate="ignore")

end_time = time.time()
print(f"\n🎉 Added {count} labeled papers in {end_time - start_time:.2f} seconds.")

In [ ]:
# @title 🧹 Remove "decision" and "dataset" columns
# This script iterates through Papers and deletes the specific attributes.

# 1. Define the AQL Query
# We filter for papers that actually have these fields to save processing time.
# Setting them to 'null' with 'keepNull: false' deletes the attribute entirely.
aql_clean_columns = """
FOR p IN Papers
  FILTER HAS(p, "decision") OR HAS(p, "dataset")

  UPDATE p WITH {
    decision: null,
    dataset: null
  } IN Papers
  OPTIONS { keepNull: false }
"""

# 2. Execute
print("Removing 'decision' and 'dataset' columns...")
cursor = db.aql.execute(aql_clean_columns)

# 3. Verify
# Let's count if any remain
verification_query = """
FOR p IN Papers
  FILTER HAS(p, "decision")
  COLLECT WITH COUNT INTO count
  RETURN count
"""
count = list(db.aql.execute(verification_query))[0]

if count == 0:
    print("✅ Success! Columns removed. All papers now have a uniform schema.")
else:
    print(f"⚠️ Something went wrong. {count} papers still have the decision column.")